In [62]:
import pandas as pd
import numpy as np 


df = pd.read_csv('matches2.csv')

In [63]:
new_df = df[['team1', 'team2', 'winner', 'toss_decision', 'toss_winner']]

In [64]:
new_df.dropna(inplace=True)
#drop Nan values
#returns new DataFrame object the inplace parameter is not set to True

C:\Users\KD\AppData\Local\Temp\ipykernel_8376\862249462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


In [65]:
X = new_df[['team1', 'team2', 'toss_decision', 'toss_winner']]
y = new_df[['winner']]

In [66]:
all_teams = {}
cnt = 0
for i in range(len(df)):
    if df.loc[i]['team1'] not in all_teams:
        all_teams[df.loc[i]['team1']] = cnt
        cnt += 1    
        
    if df.loc[i]['team2'] not in all_teams:
        all_teams[df.loc[i]['team2']] = cnt
        cnt += 1
print(all_teams)

{'Sunrisers Hyderabad': 0, 'Royal Challengers Bangalore': 1, 'Mumbai Indians': 2, 'Rising Pune Supergiant': 3, 'Gujarat Lions': 4, 'Kolkata Knight Riders': 5, 'Kings XI Punjab': 6, 'Delhi Daredevils': 7, 'Chennai Super Kings': 8, 'Rajasthan Royals': 9, 'Deccan Chargers': 10, 'Kochi Tuskers Kerala': 11, 'Pune Warriors': 12, 'Rising Pune Supergiants': 13, 'Delhi Capitals': 14}


In [67]:
encoded_teams = {w:k for k,w in all_teams.items()}

In [68]:
X = np.array(X)
y = np.array(y)

for i in range(len(X)):
    X[i][0] = all_teams[X[i][0]]
    X[i][1] = all_teams[X[i][1]]
    X[i][3] = all_teams[X[i][3]]

    y[i][0] = all_teams[y[i][0]]

print(X)



[[0 1 'field' 1]
 [2 3 'field' 3]
 [4 5 'field' 5]
 ...
 [0 14 'field' 14]
 [14 8 'field' 8]
 [2 8 'bat' 2]]


In [69]:
fb = {'field':0, 'bat':1}

for i in range (len(X)):
    X[i][2] = fb[X[i][2]]

print(fb)

{'field': 0, 'bat': 1}


In [70]:
for i in range(len(X)):
    if X[i][3] == X[i][0]:
        X[i][3] = 0
    else :
        X[i][3] = 1

print(['team1', 'team2', 'toss_decision', 'toss_winner'])
print(X)


['team1', 'team2', 'toss_decision', 'toss_winner']
[[0 1 0 1]
 [2 3 0 1]
 [4 5 0 1]
 ...
 [0 14 0 1]
 [14 8 0 1]
 [2 8 1 0]]


In [71]:
ones = 0
for i in range(len(y)):
    if y[i][0] == X[i][1] :
        if ones <= 370:
            ones += 1
            y[i][0] = 1
        else:
            t = X[i][0]
            X[i][0] = X[i][1] 
            X[i][1] = t

            y[i][0] = 0
    else :
        y[i][0] = 0

print(y)

[[0]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [72]:
X = np.array(X, dtype='int32')
y = np.array(y, dtype='int32')

y = y.ravel()

print(np.unique(y, return_counts = True))


(array([0, 1]), array([1133,  371], dtype=int64))


In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.1)

In [74]:

from sklearn.svm import SVC
model1 = SVC().fit(X_train, y_train)
model1.score(X_test, y_test)





0.7483443708609272

In [79]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier().fit(X_train, y_train)
model2.score(X_test, y_test)

0.695364238410596

In [76]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=250).fit(X, y)
model3.score(X_test, y_test)

0.7615894039735099

In [77]:
test = np.array([1,2, 0, 0]).reshape(1,-1)
model1.predict(test)
model2.predict(test)
model3.predict(test)


array([0])

In [78]:
import pickle as pkl

with open('model.pkl', 'wb') as f:
    pkl.dump(model3, f)

with open('vocak.pkl', 'wb') as f:
    pkl.dump(encoded_teams, f)

with open('inv_vocab.pkl', 'wb') as f:
    pkl.dump(all_teams, f)